In [31]:
# Running for the first time? Install Keras:
#!pip install keras
#!pip install tensorflow-gpu --yes
#!pip install opencv-python

In [11]:
import sklearn
import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50
#from tensorflow.keras.applications.inception_v3 import InceptionV3
import cv2
from PIL import Image
import pandas as pd

In [12]:
def read_process_image(path, transformation):
    """
    Reads an image and performs the desired transformation
    """
    img = cv2.imread(path,cv2.IMREAD_COLOR)
    rows,cols, channels = img.shape

    if transformation == 'translate':
        x_translate = 1
        y_translate = 1
        M = np.float32([[1,0,x_translate],[0,1,y_translate]]) # translation
        res = cv2.warpAffine(img,M,(cols,rows))
        
    if transformation == 'scale':
        scale_x = 1 # Reduce x by 1 pixel
        scale_y = 1
        res = cv2.resize(img,(cols - scale_x, rows - scale_y), interpolation = cv2.INTER_LINEAR)

    res = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
    res = Image.fromarray(res)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    return img, res

In [13]:
def preds_comparison(pred1, pred2):
    """
    Calculates metrics based on two sets of predictions.
    """
    k = len(pred1)
    if k < 1:
        return
    if pred1[0][0] == pred2[0][0]:
        top1_changed = 0
    else:
        top1_changed = 1
        
    top1_delta = abs(pred1[0][2] - pred2[0][2])
    
    avg_delta = abs(sum([abs(pred1[i][2] - pred2[i][2]) for i in range(k)]) / k)
    return top1_changed, top1_delta, avg_delta

In [14]:
def predict(model, img, img_transformed):
    """
    Given a model, an image and a transformed image, generate the predictions
    """
    k = 3
    # Original
    x = np.expand_dims(img, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)
    print('Original:', decode_predictions(preds, top=3)[0])
    pred_orig = decode_predictions(preds, top=k)[0]
    
    # Transformed 
    x = np.expand_dims(img_transformed, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)
    print('Transformed:', decode_predictions(preds, top=k)[0])
    pred_trans = decode_predictions(preds, top=k)[0]
    top1_class_pred = pred_trans[0][1]
    
    top1_changed, top1_delta, avg_delta = preds_comparison(pred_orig, pred_trans)
    return top1_changed, top1_delta, avg_delta, top1_class_pred

In [16]:
model = ResNet50(weights='imagenet')

In [29]:
output_df = pd.DataFrame([],
                   columns=['Class Name', 'Pred Class Name', 'Top 1 Changed', 'Top 1 Delta', 'Avg Delta', 'Path'])
for i in range(100):
    image_path = 'elephant2.jpeg'
    class_nm = 'Indian_elephant'
    img, res = read_process_image('elephant2.jpeg', 'translate')
    top1_changed, top1_delta, avg_delta, top1_class_pred = predict(model, img, res)
    row_append = np.array([class_nm, top1_class_pred, top1_changed, top1_delta, avg_delta, image_path])
    output_df = output_df.append(dict(zip(output_df.columns, row_append)), ignore_index=True)

Original: [('n02504013', 'Indian_elephant', 0.5157266), ('n02504458', 'African_elephant', 0.24290767), ('n01871265', 'tusker', 0.19661099)]
Transformed: [('n02504013', 'Indian_elephant', 0.56252706), ('n02504458', 'African_elephant', 0.24238445), ('n01871265', 'tusker', 0.14113507)]
Original: [('n02504013', 'Indian_elephant', 0.5157266), ('n02504458', 'African_elephant', 0.24290767), ('n01871265', 'tusker', 0.19661099)]
Transformed: [('n02504013', 'Indian_elephant', 0.56252706), ('n02504458', 'African_elephant', 0.24238445), ('n01871265', 'tusker', 0.14113507)]
Original: [('n02504013', 'Indian_elephant', 0.5157266), ('n02504458', 'African_elephant', 0.24290767), ('n01871265', 'tusker', 0.19661099)]
Transformed: [('n02504013', 'Indian_elephant', 0.56252706), ('n02504458', 'African_elephant', 0.24238445), ('n01871265', 'tusker', 0.14113507)]
Original: [('n02504013', 'Indian_elephant', 0.5157266), ('n02504458', 'African_elephant', 0.24290767), ('n01871265', 'tusker', 0.19661099)]
Transfor

Original: [('n02504013', 'Indian_elephant', 0.5157266), ('n02504458', 'African_elephant', 0.24290767), ('n01871265', 'tusker', 0.19661099)]
Transformed: [('n02504013', 'Indian_elephant', 0.56252706), ('n02504458', 'African_elephant', 0.24238445), ('n01871265', 'tusker', 0.14113507)]
Original: [('n02504013', 'Indian_elephant', 0.5157266), ('n02504458', 'African_elephant', 0.24290767), ('n01871265', 'tusker', 0.19661099)]
Transformed: [('n02504013', 'Indian_elephant', 0.56252706), ('n02504458', 'African_elephant', 0.24238445), ('n01871265', 'tusker', 0.14113507)]
Original: [('n02504013', 'Indian_elephant', 0.5157266), ('n02504458', 'African_elephant', 0.24290767), ('n01871265', 'tusker', 0.19661099)]
Transformed: [('n02504013', 'Indian_elephant', 0.56252706), ('n02504458', 'African_elephant', 0.24238445), ('n01871265', 'tusker', 0.14113507)]
Original: [('n02504013', 'Indian_elephant', 0.5157266), ('n02504458', 'African_elephant', 0.24290767), ('n01871265', 'tusker', 0.19661099)]
Transfor

Original: [('n02504013', 'Indian_elephant', 0.5157266), ('n02504458', 'African_elephant', 0.24290767), ('n01871265', 'tusker', 0.19661099)]
Transformed: [('n02504013', 'Indian_elephant', 0.56252706), ('n02504458', 'African_elephant', 0.24238445), ('n01871265', 'tusker', 0.14113507)]
Original: [('n02504013', 'Indian_elephant', 0.5157266), ('n02504458', 'African_elephant', 0.24290767), ('n01871265', 'tusker', 0.19661099)]
Transformed: [('n02504013', 'Indian_elephant', 0.56252706), ('n02504458', 'African_elephant', 0.24238445), ('n01871265', 'tusker', 0.14113507)]
Original: [('n02504013', 'Indian_elephant', 0.5157266), ('n02504458', 'African_elephant', 0.24290767), ('n01871265', 'tusker', 0.19661099)]
Transformed: [('n02504013', 'Indian_elephant', 0.56252706), ('n02504458', 'African_elephant', 0.24238445), ('n01871265', 'tusker', 0.14113507)]
Original: [('n02504013', 'Indian_elephant', 0.5157266), ('n02504458', 'African_elephant', 0.24290767), ('n01871265', 'tusker', 0.19661099)]
Transfor

Original: [('n02504013', 'Indian_elephant', 0.5157266), ('n02504458', 'African_elephant', 0.24290767), ('n01871265', 'tusker', 0.19661099)]
Transformed: [('n02504013', 'Indian_elephant', 0.56252706), ('n02504458', 'African_elephant', 0.24238445), ('n01871265', 'tusker', 0.14113507)]
Original: [('n02504013', 'Indian_elephant', 0.5157266), ('n02504458', 'African_elephant', 0.24290767), ('n01871265', 'tusker', 0.19661099)]
Transformed: [('n02504013', 'Indian_elephant', 0.56252706), ('n02504458', 'African_elephant', 0.24238445), ('n01871265', 'tusker', 0.14113507)]
Original: [('n02504013', 'Indian_elephant', 0.5157266), ('n02504458', 'African_elephant', 0.24290767), ('n01871265', 'tusker', 0.19661099)]
Transformed: [('n02504013', 'Indian_elephant', 0.56252706), ('n02504458', 'African_elephant', 0.24238445), ('n01871265', 'tusker', 0.14113507)]
Original: [('n02504013', 'Indian_elephant', 0.5157266), ('n02504458', 'African_elephant', 0.24290767), ('n01871265', 'tusker', 0.19661099)]
Transfor